In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../../MNIST_data/", one_hot=True)

W0221 23:22:46.587664 4589438400 deprecation.py:323] From <ipython-input-1-8bf8ae5a5303>:2: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0221 23:22:46.588807 4589438400 deprecation.py:323] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0221 23:22:46.589628 4589438400 deprecation.py:323] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will 

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz


W0221 23:22:46.817062 4589438400 deprecation.py:323] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
import tensorflow as tf
import numpy as np
np.set_printoptions(suppress=True) 
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

In [3]:
mb_size =128
Z_dim = 100
X_dim = mnist.train.images.shape[1]
y_dim = mnist.train.labels.shape[1]

def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1./tf.sqrt(in_dim/2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

X = tf.placeholder(tf.float32, shape=[None, X_dim])
y = tf.placeholder(tf.float32, shape=[None, y_dim])

D_W1 = tf.Variable(xavier_init([X_dim + y_dim, 128]))
D_b1 = tf.Variable(tf.zeros(shape=[128]))

D_W2 = tf.Variable(xavier_init([128, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))

theta_D = [D_W1, D_W2, D_b1, D_b2]

def discriminator(x, y):
    inputs = tf.concat(axis=1, values=[x, y])
    D_h1 = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)
    return D_prob, D_logit

Z = tf.placeholder(tf.float32, shape=[None, Z_dim])

G_W1 = tf.Variable(xavier_init([Z_dim + y_dim, 128]))
G_b1 = tf.Variable(tf.zeros(shape=[128]))

G_W2 = tf.Variable(xavier_init([128, X_dim]))
G_b2 = tf.Variable(tf.zeros(shape=[X_dim]))

theta_G = [G_W1, G_W2, G_b1, G_b2]

def generator(z, y):
    inputs = tf.concat(axis=1, values=[z, y])
    G_h1 = tf.nn.relu(tf.matmul(inputs, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)
    return G_prob

def sample_Z(m,n):
    return np.random.uniform(-1., 1., size=[m,n])

def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)
    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')
    return fig

G_sample = generator(Z, y)
D_real, D_logit_real = discriminator(X, y)
D_fake, D_logit_fake = discriminator(G_sample, y)

D_loss = -tf.reduce_mean(tf.log(D_real + 1e-8) + tf.log(1. - D_fake + 1e-8))
G_loss = -tf.reduce_mean(tf.log(D_fake + 1e-8))

# D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
# D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))
# D_loss = D_loss_real + D_loss_fake
# G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))

D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

if not os.path.exists('out/'):
    os.makedirs('out/')
    
i=0
for it in range(1000000):
    if it%1000==0:
        for j in range(10):
            n_sample = 16

            Z_sample = sample_Z(n_sample, Z_dim)
            y_sample = np.zeros(shape=[n_sample, y_dim])
            y_sample[:,j] = 1

            samples = sess.run(G_sample, feed_dict={Z:Z_sample, y:y_sample})

            fig = plot(samples)
            plt.savefig('out/{}.png'.format((str(i)+'_'+str(j)).zfill(3)), bbox_inches='tight')
            i+=1
            plt.close(fig)
        
    X_mb, y_mb =  mnist.train.next_batch(mb_size)
    Z_sample = sample_Z(mb_size, Z_dim)
        
    _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X:X_mb, Z:Z_sample, y:y_mb})
    _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z:Z_sample, y:y_mb})
    
    if it%1000==0:
        print('iter:{}'.format(it))
        print('D loss:{:.4}'.format(D_loss_curr))
        print('G loss:{:.4}'.format(G_loss_curr))
        print()

W0221 23:22:47.222492 4589438400 deprecation.py:506] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/python/training/slot_creator.py:187: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


iter:0
D loss:1.307
G loss:2.544

iter:1000
D loss:0.005521
G loss:7.26

iter:2000
D loss:0.03372
G loss:5.611

iter:3000
D loss:0.03694
G loss:5.674

iter:4000
D loss:0.1072
G loss:4.955

iter:5000
D loss:0.2016
G loss:5.431

iter:6000
D loss:0.383
G loss:4.643

iter:7000
D loss:0.3337
G loss:4.515

iter:8000
D loss:0.4677
G loss:3.992

iter:9000
D loss:0.4661
G loss:3.147

iter:10000
D loss:0.7423
G loss:3.141

iter:11000
D loss:0.7281
G loss:2.379

iter:12000
D loss:0.8181
G loss:2.625

iter:13000
D loss:0.7116
G loss:3.008

iter:14000
D loss:0.7395
G loss:2.443

iter:15000
D loss:0.8384
G loss:1.998

iter:16000
D loss:0.7784
G loss:2.206

iter:17000
D loss:0.6645
G loss:2.289

iter:18000
D loss:0.7668
G loss:2.123

iter:19000
D loss:0.974
G loss:1.772

iter:20000
D loss:0.7069
G loss:1.945

iter:21000
D loss:0.8859
G loss:1.87

iter:22000
D loss:0.7662
G loss:1.878

iter:23000
D loss:0.8224
G loss:1.735

iter:24000
D loss:0.903
G loss:1.794

iter:25000
D loss:0.8373
G loss:1.686

i

KeyboardInterrupt: 